In [2]:
!pip install lightgbm

In [3]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split

In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-inventory-demand-forecasting-dataset/train.csv
/kaggle/input/store-inventory-demand-forecasting-dataset/test.csv


# EDA

In [5]:
pdf = pd.read_csv('/kaggle/input/store-inventory-demand-forecasting-dataset/train.csv')
pdf.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [6]:
pdf = pdf.rename(columns={
    'date': 'sales_date',
    'store': 'store_id',
    'item': 'item_id',
    'sales': 'qty_sold'
})
pdf['sales_date'] = pd.to_datetime(pdf['sales_date'])
pdf['sales_date'] = pdf['sales_date'].apply(lambda x: x.timestamp())

pdf.head()

,sales_date,store_id,item_id,qty_sold
0,1.356998e+09,1,1,13
1,1.357085e+09,1,1,11
2,1.357171e+09,1,1,14
3,1.357258e+09,1,1,13
4,1.357344e+09,1,1,10


In [7]:
pdf.dtypes

sales_date    float64
store_id        int64
item_id         int64
qty_sold        int64
dtype: object

In [8]:
pdf.shape

(913000, 4)

# Get Test Data

In [9]:
test_df = pd.read_csv('/kaggle/input/store-inventory-demand-forecasting-dataset/test.csv')

In [18]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df['date'] = test_df['date'].apply(lambda x: x.timestamp())


In [19]:
test_data = lgbm.Dataset(test_df)

# Train

In [10]:
train_data = lgbm.Dataset(pdf)

In [11]:
param = {'num_leaves': 31, 'objective': 'binary'}
param['metric'] = 'auc'


In [13]:
bst = lgbm.train(param, train_data, 10)

[LightGBM] [Warning] Contains only one class
[LightGBM] [Info] Number of positive: 0, number of negative: 913000
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 506
[LightGBM] [Info] Number of data points in the train set: 913000, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.000000 -> initscore=-34.538776
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the sp

In [20]:
bst.predict(test_df)

array([1.e-15, 1.e-15, 1.e-15, ..., 1.e-15, 1.e-15, 1.e-15])